# Auxiliando os Times de Marketing e Pricing da Empresa de Delivery

# Contexto

## [Universidade dos Dados](https://www.universidadedosdados.com/)

Este Desafio foi proposto pela  [https://www.universidadedosdados.com/](https://www.universidadedosdados.com/)


## Desafio 

Nós estamos trabalhando em uma empresa de delivery, num time de dados centralizado que atende a diversas áreas e recebemos algumas demandas:

1. Numa ação de marketing, para atrair mais entregadores, vamos dar uma bonificação para os 20 entregadores que possuem maior distância percorrida ao todo. A bonificação vai variar de acordo com o tipo de profissional que ele é e o modelo que ele usa para se locomover (moto, bike, etc). Levante essas informações.
1. Além disso, o time de Pricing precisa ajustar os valores pagos aos entregadores. Para isso, eles precisam da distribuição da distância média percorrida pelos motoqueiros separada por estado, já que cada região terá seu preço.
1. Por fim, o CFO precisa de alguns indicadores de receita para apresentar para a diretoria executiva. Dentre esses indicadores, vocês precisarão levantar (1) a receita média e total separada por tipo (Food x Good), (2) A receita média e total por estado. Ou seja, são 4 tabelas ao todo.
2. Se a empresa tem um gasto fixo de 5 reais por entrega, recebe 15% do valor de cada entrega como receita e, do total do lucro, distribui 20% em forma de bônus para os 2 mil funcionários, quanto cada um irá receber no período contido no dataset?

## Fonte do DataSet
 Kaggle: [https://www.kaggle.com/datasets/nosbielcs/brazilian-delivery-center](https://www.kaggle.com/datasets/nosbielcs/brazilian-delivery-center)

## Descrição dos datasets

- channels: Este dataset possui informações sobre os canais de venda (marketplaces) onde são vendidos os good e food de nossos lojistas.
- deliveries: Este dataset possui informações sobre as entregas realizadas por nossos entregadores parceiros.
- drivers: Este dataset possui informações sobre os entregadores parceiros. Eles ficam em nossos hubs e toda vez que um pedido é processado, são eles fazem as entregas na casa dos consumidores.
- hubs: Este dataset possui informações sobre os hubs do Delivery Center. Entenda que os Hubs são os centros de distribuição dos pedidos e é dali que saem as entregas.
- orders: Este dataset possui informações sobre as vendas processadas através da plataforma do Delivery Center.
- payments: Este dataset possui informações sobre os pagamentos realizados ao Delivery Center.
- stores: Este dataset possui informações sobre os lojistas. Eles utilizam a Plataforma do Delivery Center para vender seus itens (good e/ou food) nos marketplaces.

# Let's go 
## Setup

In [31]:
import os
import pandas as pd

## Carregando

In [32]:
# Diretório contendo os arquivos
dir_path = 'dataset'

# Inicializando um dicionário para armazenar os DataFrames
dataframes = {}

# Iterar sobre os arquivos no diretório
for filename in os.listdir(dir_path):
    if filename.endswith('.csv'):  # Filtra para arquivos CSV, mas você pode ajustar para outros tipos
        file_path = os.path.join(dir_path, filename)
        # Nome do DataFrame será o nome do arquivo sem extensão
        df_name = os.path.splitext(filename)[0]
        # Carregar o arquivo CSV no DataFrame
        # Tentar carregar o arquivo CSV no DataFrame com diferentes codificações
        try:
            dataframes[df_name] = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            # Se UTF-8 falhar, tentar com outra codificação
            dataframes[df_name] = pd.read_csv(file_path, encoding='ISO-8859-1')
    

In [3]:
chaves = dataframes.keys()

print(chaves)

for chave in chaves:
    print(f'+++++{chave}++++++')
    print(dataframes[chave].head())
    print(dataframes[chave].shape)
    

dict_keys(['stores', 'payments', 'channels', 'hubs', 'drivers', 'orders', 'deliveries'])
+++++stores++++++
   store_id  hub_id         store_name store_segment  store_plan_price  \
0         3       2            CUMIURI          FOOD               0.0   
1         6       3  PIMGUCIS DA VIVA           FOOD               0.0   
2         8       3          RASMUR S           FOOD               0.0   
3        53       8        PAPA SUCIS           FOOD               0.0   
4        54       8      VUZPI PAZZIS           FOOD               0.0   

   store_latitude  store_longitude  
0             NaN              NaN  
1      -30.037415       -51.203520  
2      -30.037415       -51.203520  
3      -22.921475       -43.234822  
4      -22.921475       -43.234822  
(951, 7)
+++++payments++++++
   payment_id  payment_order_id  payment_amount  payment_fee payment_method  \
0     4427917          68410055          118.44         0.00        VOUCHER   
1     4427918          68410055        

## Questão 1

Em uma ação de marketing para atrair mais entregadores, iremos oferecer uma bonificação aos 20 entregadores que percorreram a maior distância total. O valor da bonificação será ajustado de acordo com o tipo de entregador e o meio de transporte que utilizam (moto, bicicleta, etc). Levante as informações necessárias para identificar esses profissionais e calcular a bonificação correspondente.

Para responder a essa pergunta, utilizaremos os seguintes datasets:

- **`drivers`** com as colunas:
    - **`driver_modal`**: Representa o meio de transporte utilizado pelo entregador.
    - **`driver_id`**: Identificação única do entregador.






- **`deliveries`** com as colunas:
    - **`driver_id`**: Identificação única do entregador (relacionada ao dataset `drivers`).
    - **`delivery_distance_meters`**: Distância percorrida durante a entrega, medida em metros.
    - **`delivery_status`**: Status da entrega. 

In [41]:
df = pd.merge(dataframes['drivers'][['driver_id', 'driver_modal']], dataframes['deliveries'][['driver_id', 'delivery_distance_meters', 'delivery_status']], on='driver_id')

In [42]:
df.head()

,driver_id,driver_modal,delivery_distance_meters,delivery_status
0,133,MOTOBOY,2188.0,DELIVERED
1,133,MOTOBOY,1296.0,DELIVERED
2,133,MOTOBOY,2402.0,DELIVERED
3,133,MOTOBOY,1871.0,DELIVERED
4,133,MOTOBOY,1309.0,DELIVERED


Vamos considerar que, no edital, apenas as entregas concluídas sejam levadas em conta.

In [61]:
df = df[df['delivery_status'] == 'DELIVERED']

In [62]:
df['driver_modal'].isnull().sum()

0

In [63]:
df['driver_modal'].value_counts()

driver_modal
MOTOBOY    265294
BIKER       97472
Name: count, dtype: int64

In [64]:
df_moto = df[ df['driver_modal'] == 'MOTOBOY']

In [54]:
df_moto.describe()

,driver_id,delivery_distance_meters
count,265443.000000,2.654180e+05
mean,21438.995366,3.447114e+03
std,15575.996963,2.167472e+04
min,133.000000,0.000000e+00
25%,7754.000000,1.739000e+03
50%,20495.000000,2.615000e+03
75%,31126.000000,3.972000e+03
max,66459.000000,7.251291e+06


In [59]:
possivel_fraude = df_moto[df_moto['delivery_distance_meters'] > 7700 ]
possivel_fraude.shape

(12552, 4)

In [52]:
# Agrupar por 'id' e somar as distâncias
df_grouped = df_moto.groupby('driver_id', as_index=False)['delivery_distance_meters'].sum()
df_grouped.head()

,driver_id,delivery_distance_meters
0,133,45188.0
1,138,323114.0
2,140,1462661.0
3,148,1253116.0
4,165,41258.0


In [53]:

df_grouped.sort_values(by='delivery_distance_meters', ascending=False).head(20)

,driver_id,delivery_distance_meters
1332,26223,60504610.0
1294,25651,32466086.0
896,15898,8209339.0
982,17749,8060048.0
89,731,7199476.0
1606,31023,6096369.0
2107,42146,4845699.0
686,11063,4829062.0
813,14136,4320123.0
43,464,4247737.0


In [45]:
df.describe()

,driver_id,delivery_distance_meters
count,362957.000000,3.629280e+05
mean,21001.777643,2.818972e+03
std,15520.652243,1.874356e+04
min,133.000000,0.000000e+00
25%,7615.000000,1.172000e+03
50%,18754.000000,2.044000e+03
75%,31050.000000,3.419000e+03
max,66459.000000,7.251291e+06
